In [10]:
import numpy as np
import rasterio as rs
from rasterio.mask import mask
import requests
from io import BytesIO
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

# Function to reproject raster data
def reproject_raster(src, target_crs):
    src_data = src.read(1)  # Read the first band
    dst_data = np.empty(src_data.shape, dtype=np.float32)

    # Perform the reprojection
    reproject(
        source=src_data,
        destination=dst_data,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=src.transform,
        dst_crs=target_crs,
        resampling=Resampling.nearest)

    return dst_data, src.transform, target_crs


# path to shape file:

mask_paths = [ 'CANARC_BS_LancasterSound_Province01.shp','CANARC_BS_LancasterSound_Province02.shp','CANARC_BS_LancasterSound_Province03.shp']
site_path = 'canarc_bs_loc.shp'

mask_names = ['LS_01', 'LS_02', 'LS_03']

# years for processing:

years = ['2013','2014','2015','2016','2017','2018']
process_months = [1,8] #Edit as needed if you want to only process specific months

# Load shape files for the sites
site_path = 'canarc_bs_loc.shp'
site_gpd = gpd.read_file(site_path)


#Start loop to go over years and masks:

for year in years:
    for mask_path, mask_name in zip(mask_paths, mask_names):
        # Load the mask
        mask_gpd = gpd.read_file(mask_path)
        mask_gpd = mask_gpd.to_crs('EPSG:3413')  # Match the projections to the sea ice
        bs_mask = mask_gpd.iloc[0].geometry

        # Initialize a dictionary to store mean values
        mean_values_by_date = {}

 # Initialize a dictionary to store mean values
        mean_values_by_date = {}

        # Assuming non-leap year; adjust February to 29 for leap years.
        months_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        months_str = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

        # Loop through each month and day:
        
        for month_index, days_in_month in enumerate(months_days):
            month_int = month_index + 1 
            if month_int in process_months: #Will filter for the specified months.
                month_str = months_str[month_index]
            
            for day in range(1, days_in_month + 1):
                date = f'{year}{month_int:02d}{day:02d}'
                url = f'https://data.seaice.uni-bremen.de/amsr2/asi_daygrid_swath/n3125/{year}/{month_str}/Arctic3125/asi-AMSR2-n3125-{date}-v5.4.tif'
                print(f'Accessing data for {date}: {url}')

        def reproject_raster(src, target_crs):
        
                # Request and process data for each day
                response = requests.get(url)
                if response.status_code == 200:
                    today_seaice_raw = rs.open(BytesIO(response.content))
                    today_seaice_3995, transform_3995, crs_3995 = reproject_raster(today_seaice_raw, 'EPSG:3995')
        
                    # Apply the masks
                    masked_bs, transform_bs = mask(today_seaice_raw, [bs_mask], pad=False, crop=True, nodata=255)
            
                    # Calculate mean sea ice concentration, excluding land and mask values
                    bs_mean = np.mean(masked_bs[0][(masked_bs[0] != 255) & (masked_bs[0] !=120)] )
        
                    mean_values_by_date[date] = {'bs_mean': bs_mean}
                else:
                    print(f'Failed to download data for {date}')
                    mean_values_by_date[date] = {'bs_mean': np.nan, 'bs_mean': np.nan}


                
                # response = requests.get(url)
                # if response.status_code == 200:
                #     today_seaice_raw = rs.open(BytesIO(response.content))
                #     masked_bs, transform_bs = mask(today_seaice_raw, [bs_mask], pad=False, crop=True, nodata=255)

                #     # Calculate mean sea ice concentration, excluding land (120) and mask(255) values
                #     bs_mean = np.mean(masked_bs[0][(masked_bs[0] != 255) & (masked_bs[0] !=120)])
                #     mean_values_by_date[date] = {'bs_mean': bs_mean}
                # else:
                #     print(f'Failed to download data for {date}')
                #     mean_values_by_date[date] = {'bs_mean': np.nan}

        # Convert the dictionary to a DataFrame and plot
        df = pd.DataFrame.from_dict(mean_values_by_date, orient='index', columns=['bs_mean'])
        df.index = pd.to_datetime(df.index)

        #Save csv:

        csv_path = f'E:/CANARC_All/ice_data/CANARC_BS/LS_poly/CANARC_BS_{year}_{mask_name}_NorthWestPassage_3125Km.csv'
        df.to_csv(csv_path, index=True, index_label='Date')
        print(f'Data saved to {csv_path}')

        #Plot and save ice concentrations:

        plt.figure(figsize=(10, 6))
        plt.plot(df.index, df['bs_mean'], label='Daily mean', marker='o')
        plt.title(f'Daily Sea Ice Concentration in {year} for {mask_name}')
        plt.xlabel('Date')
        plt.ylabel('Mean Sea Ice %')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'E:/CANARC_All/ice_data/CANARC_BS/LS_poly/CANARC_BS_{year}_{mask_name}_MeanIceConcentration.png')
        plt.close()





Accessing data for 20130101: https://data.seaice.uni-bremen.de/amsr2/asi_daygrid_swath/n3125/2013/jan/Arctic3125/asi-AMSR2-n3125-20130101-v5.4.tif
Accessing data for 20130102: https://data.seaice.uni-bremen.de/amsr2/asi_daygrid_swath/n3125/2013/jan/Arctic3125/asi-AMSR2-n3125-20130102-v5.4.tif
Accessing data for 20130103: https://data.seaice.uni-bremen.de/amsr2/asi_daygrid_swath/n3125/2013/jan/Arctic3125/asi-AMSR2-n3125-20130103-v5.4.tif
Accessing data for 20130104: https://data.seaice.uni-bremen.de/amsr2/asi_daygrid_swath/n3125/2013/jan/Arctic3125/asi-AMSR2-n3125-20130104-v5.4.tif
Accessing data for 20130105: https://data.seaice.uni-bremen.de/amsr2/asi_daygrid_swath/n3125/2013/jan/Arctic3125/asi-AMSR2-n3125-20130105-v5.4.tif
Accessing data for 20130106: https://data.seaice.uni-bremen.de/amsr2/asi_daygrid_swath/n3125/2013/jan/Arctic3125/asi-AMSR2-n3125-20130106-v5.4.tif
Accessing data for 20130107: https://data.seaice.uni-bremen.de/amsr2/asi_daygrid_swath/n3125/2013/jan/Arctic3125/asi-A

In [13]:
import numpy as np
import rasterio as rs
from rasterio.mask import mask
import requests
from io import BytesIO
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from rasterio.warp import reproject, Resampling

# Function to reproject raster data
def reproject_raster(src, target_crs):
    src_data = src.read(1)  # Read the first band
    dst_data = np.empty(src_data.shape, dtype=np.float32)
    # Perform the reprojection
    reproject(
        source=src_data,
        destination=dst_data,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=src.transform,
        dst_crs=target_crs,
        resampling=rs.warp.Resampling.nearest)
    return dst_data, src.transform, target_crs

# Define paths and names
mask_paths = ['CANARC_BS_LancasterSound_Province01.shp', 'CANARC_BS_LancasterSound_Province02.shp', 'CANARC_BS_LancasterSound_Province03.shp']
mask_names = ['LS_01', 'LS_02', 'LS_03']
years = ['2013', '2014', '2015', '2016', '2017', '2018']
process_months = [1, 8]

# Load shape files for the sites
site_gpd = gpd.read_file('canarc_bs_loc.shp')

# Loop over years and masks
for year in years:
    for mask_path, mask_name in zip(mask_paths, mask_names):
        mask_gpd = gpd.read_file(mask_path)
        mask_gpd = mask_gpd.to_crs('EPSG:3413')
        bs_mask = mask_gpd.iloc[0].geometry
        mean_values_by_date = {}
        months_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        months_str = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

        for month_index, days_in_month in enumerate(months_days):
            month_int = month_index + 1
            if month_int in process_months:
                month_str = months_str[month_index]
                for day in range(1, days_in_month + 1):
                    date = f'{year}{month_int:02d}{day:02d}'
                    url = f'https://data.seaice.uni-bremen.de/amsr2/asi_daygrid_swath/n3125/{year}/{month_str}/Arctic3125/asi-AMSR2-n3125-{date}-v5.4.tif'
                    print(f'Accessing data for {date}: {url}')
                    response = requests.get(url)
                    if response.status_code == 200:
                        with rs.open(BytesIO(response.content)) as today_seaice_raw:
                            today_seaice_3995, transform_3995, crs_3995 = reproject_raster(today_seaice_raw, 'EPSG:3995')
                            masked_bs, transform_bs = mask(today_seaice_raw, [bs_mask], pad=False, crop=True, nodata=255)
                            bs_mean = np.mean(masked_bs[0][(masked_bs[0] != 255) & (masked_bs[0] != 120)])
                            mean_values_by_date[date] = {'bs_mean': bs_mean}
                    else:
                        print(f'Failed to download data for {date}')
                        mean_values_by_date[date] = {'bs_mean': np.nan}
                        
 # Create DataFrame, save to CSV, and plot
        df = pd.DataFrame.from_dict(mean_values_by_date, orient='index', columns=['bs_mean'])
        df.index = pd.to_datetime(df.index)
        csv_path = f'E:/CANARC_All/ice_data/CANARC_BS/LS_poly/CANARC_BS_{year}_{mask_name}_NorthWestPassage_3125Km.csv'
        df.to_csv(csv_path, index=True, index_label='Date')
        print(f'Data saved to {csv_path}')
        plt.figure(figsize=(10, 6))
        plt.plot(df.index, df['bs_mean'], label='Daily mean', marker='o')
        plt.title(f'Daily Sea Ice Concentration in {year} for {mask_name}')
        plt.xlabel('Date')
        plt.ylabel('Mean Sea Ice %')
        plt.legend()
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'E:/CANARC_All/ice_data/CANARC_BS/LS_poly/CANARC_BS_{year}_{mask_name}_MeanIceConcentration.png')
        plt.close()

Accessing data for 20130101: https://data.seaice.uni-bremen.de/amsr2/asi_daygrid_swath/n3125/2013/jan/Arctic3125/asi-AMSR2-n3125-20130101-v5.4.tif


TypeError: argument of type 'NoneType' is not iterable